In [2]:
from lxml import etree
import pandas as pd
import numpy as np
import random
import re,requests,time
import urllib
import lxml
from tqdm import tqdm
import json
import requests
from urllib import error
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
#from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime
from lxml import etree
import requests
import time
import re
import pandas as pd
import random

In [25]:
import requests
import json
df = pd.DataFrame(columns=['raisefund_no', 'channel'])
# 网站上的页面范围为127页
for i in range(128):
    url = 'https://partner-gateway.qschou.com/official/pc/project/succeed?page={}'.format(i)
    response = requests.get(url)
    data = response.json()
    # 提取uuid字段
    for item in data['data']:
        uuid = item["raisefund_no"] 
        channel=item["channel"] 
        df = df.append({'raisefund_no': uuid, 'channel': channel}, ignore_index=True)

In [17]:
len(uuids)

2552

In [27]:
df.to_excel('轻松筹网站id.xlsx',index=False)

In [36]:
# 项目进展披露详尽性
def get_rea(Channel,ID):
#     for i in range(len(ID)):
    url = 'https://partner-gateway.qschou.com/{}/project/{}/feed'.format(Channel,ID)
    # headers = {'Host': 'partner-gateway.qschou.com', 'Connection': 'keep-alive', 'Content-Length': '23142', 'cookie': '', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36', 'charset': 'utf-8', 'Accept-Encoding': 'gzip,compress,br,deflate', 'Content-Type': 'application/json; charset=UTF-8','Referer': 'https://qsgy.qschou.com/'}
    response = requests.get(url)
    data = response.json()
    image_count = 0
    content_count=0
    up_times=0
    if data["data"]["topics"] is not None:
        for topic in data["data"]["topics"]:
    #       更新图片数量
            if topic["image"] is not None:
                image_count += len(topic["image"])
    #       更新字数
            if topic["content"] is not None:
                content_count += len(topic["content"])
#   进展更新次数
            up_times=len(data["data"]["topics"])
    return up_times,image_count,content_count

In [49]:
# 项目基本信息
alldata=pd.DataFrame()
for i in range(1425,len(df)):
    url = 'https://partner-gateway.qschou.com/{}/project/{}/info'.format(df['channel'][i],df['raisefund_no'][i])
    response = requests.get(url)
    data = response.json()
#     项目标题
    title = data['data']['title']
#     筹资规模
    total_amount = data['data']['total_amount']
#     已筹金额
    current_amount = data['data']['current_amount']
#     捐款人数
    backer_count = data['data']['backer_count']
#     转发次数
    share_count = data['data']['share_count']
#     融资效率
    progress = data['data']['progress']
#     项目介绍
    introduction = data['data']['introduction']
    detail= data['data']['detail']
    start_index = introduction.find('起止日期')
    text_before = introduction[:start_index]
#     正文字数
    word_count = len(text_before)
#     正文图片数量
    pic= len(re.findall(r'<img', detail))
#     融资期限
    match = re.search(r'\d{4}-\d{1,2}-\d{1,2}至\d{4}-\d{1,2}-\d{1,2}',introduction)
    if match:
        date_text = match.group()
        start_date, end_date = map(datetime.strptime, date_text.split('至'), ['%Y-%m-%d']*2)
    else:
        date_text = None
    # 转换为日期对象
    # 计算持续时间
    duration = (end_date - start_date).days
    up_times,image_count,content_count=get_rea(df['channel'][i],df['raisefund_no'][i])
    data=pd.DataFrame({'项目id':[df['raisefund_no'][i]],'项目标题': title,'筹资规模': total_amount,'已筹金额': current_amount,'融资效率': progress,'捐款人数': backer_count,'转发次数': share_count,'介绍文本':text_before,'正文字数': word_count,'图片数量': pic,'融资期限':duration,'项目进展更新次数':up_times,'更新字数':content_count,'更新图片数量':image_count})
    alldata=pd.concat([alldata,data],ignore_index=True)

In [50]:
alldata

,项目id,项目标题,筹资规模,已筹金额,融资效率,捐款人数,转发次数,介绍文本,正文字数,图片数量,融资期限,项目进展更新次数,更新字数,更新图片数量
0,201902010000000039885143,大学生罹患血癌 病房中的第二个春节,500000,14870.0,2.97,487,655,我是谁今年是我和父母在医院里过的第二个春节。我叫何志铭，今年24岁，家住福建省漳州市漳浦县官...,1180,5,90,2,60,2
1,201902180000000041405295,血癌小学霸渴望移植救命 重回校园,400000,18698.0,4.67,799,673,我们的故事我的女儿思思（化名）今年9岁，她聪明、可爱、好学，在班上一直都是名列前茅。但是如今...,971,5,89,3,251,3
2,201902210000000041875342,漫长抗癌求医路 移植获新生不舍功亏一篑,300000,41404.0,13.80,1704,700,幸福的家庭，懂事的儿子我们家乡是广东省汕尾市陆丰市博美镇，现在广东省中山市的一个小镇开着一间...,1328,5,89,1,5,0
3,201902250000000042205375,顶梁柱患重病 急需移植救命,600000,38123.0,6.35,2383,923,幸福的家庭被疾病摧毁尊敬社会爱心人士您们好！我叫俞荔聪，家住福建省莆田市城厢区。我原本有一个...,1086,6,89,1,5,0
4,201902270000000042385393,百万血癌宝宝 移植后遭遇并发症,300000,13109.0,4.37,398,807,猝不及防的血癌的噩耗我是湖南省岳阳市人，我的儿子浩浩今年4岁，2017年01月28日，两岁的...,732,5,89,2,59,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,202204200000000120963262,3岁血癌宝宝盼救助,300000,36224.0,12.07,2714,84,患儿“浩浩”（化名）今年刚满3岁，家住江苏省东台市。浩浩是家里最小的儿子，上面还有个16岁的...,1863,4,653,1,20,0
1123,202205100000000121703336,单亲妈妈打三份工难救血癌儿子,300000,25552.0,8.52,1440,54,抗白4年身兼多职，只为儿子活下去“他才12岁，本来应该像其他孩子一样在学校上学念书，但是现在...,1805,4,653,1,20,0
1124,202208180000000124653631,助力生命 你我同行,1000000,321801.0,32.18,16777,226,截止今天，本项目累计筹款321801元。共救助14名患者，感谢大家的爱心捐赠,38,0,653,2,41,13
1125,202212120000000127063872,为白血病患者带来希望,1000000,11297.0,1.13,155,645,截至今日，该项目累计筹款11297元。共救助一位患者，感谢大家的爱心支持,36,0,653,0,0,0


In [51]:
alldata.to_csv('轻松筹数据科学.csv',encoding='utf_8_sig',mode='a',header=True,index=None)